In [1]:
import yaml
import os
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate

with open('config.yml', 'r') as file:
    config = yaml.safe_load(file)
openai_key = config["openai_key"]
anthropic_key = config["anthropic_key"]

In [2]:
os.environ["ANTHROPIC_API_KEY"] = anthropic_key

chat = ChatAnthropic(temperature=0, model_name="claude-3-haiku-20240307")
system = (
    """You are an AI language model tasked to beat the Wiki Game. 
    Your goal is, from a wikipedia page, to get the next link to click to go to the next page."""
)

In [7]:
from wikipedia_functions import get_random_page, get_page_content

start_page = get_random_page(1)
final_page = "United States"
print(start_page)
final_page_content = get_page_content(final_page)

['Hørring cabinet']


In [8]:
from prompt import get_crawler_template

crawler_template = get_crawler_template()

In [9]:
from wikipedia_functions import get_page_content

start_links = get_page_content(start_page)

KeyError: 'revisions'

In [11]:
import re 
import random

human = crawler_template
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])
chain = prompt | chat
current_page = start_page
current_links = start_links
for i in range(10):
    text = chain.invoke(
        {
            "current_page": current_page,
            "current_links": current_links,
            "end_page": final_page,
            "end_page_content": final_page_content
        }
    )
    model_output = text.content
    print(model_output)
    pattern = r'<output>(.*?)</output>'
    matches = re.findall(pattern, model_output)
    current_page = matches[0]
    if current_page == "Random":
        current_page = random.choice(current_links)
        print(current_page)
    if current_page != final_page:
        current_links = get_page_content(current_page)
    else:
        print(f"Page reached in {i} iterations!")
        break

<reasoning>
The goal page is "United States", and the available links are not directly related to the United States. The links are about Fiji, Bau Island, and Tailevu Province, which are not relevant to reaching the goal page. Since there are no links that could be used to get closer to the goal page, the best option is to choose a random link.
</reasoning>

<output>Random</output>
Bau Island
<reasoning>The link 'Bau (island)' is the only one available, and it does not seem to be directly related to the goal page 'United States'. Therefore, I will output "Random" as the link choice.</reasoning>
<output>Random</output>
Bau (island)
<reasoning>
The goal page is "United States", and the available links do not seem to be directly related to that topic. The links cover a wide range of topics, from Fijian language to Christianity, cannibalism, and various islands and regions in Fiji. None of these links appear to be closely related to the United States. Therefore, I will output "Random" as t

KeyError: 0